In [ ]:
# | label: setup
# | echo: false

import difflib as dl
import re
from pathlib import Path

import duckdb
import numpy as np
import pandas as pd
import plotly.express as px
from ydata_profiling import ProfileReport

from IPython.display import Markdown

In [ ]:
from itables import init_notebook_mode  # Display dataframes in a friendly manner

init_notebook_mode(all_interactive=True)

In [ ]:
# | label: data_source
# | echo: false

DATA_URL = "https://hbiostat.org/data/repo/titanic3.csv"  # hopefully this is a "definitive" source

### Source data - "1999 Original" (`titanic3`)

The source data has moved a few times:

- TODO

The latest incarnation can be found here:

- https://hbiostat.org/data/
- https://hbiostat.org/data/repo/titanic
- https://hbiostat.org/data/repo/titanic3.csv

and has been replicated countless times including the Kaggle version.


Is it possible to find actual date of birth for each passanger? These data sources only have age (it seems).

These data sets were downloaded from https://www.kaggle.com/c/titanic/data

#### DuckDB setup

In [ ]:
def load_data_duckdb(table_name="raw_titanic", data_url=DATA_URL):
    load_data_sql = f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM read_csv_auto('{data_url}')"
    con = duckdb.connect()
    con.sql(load_data_sql)
    return con

In [ ]:
def load_sql_query(file_path):
    """Load the contents of a .sql file if it exists.

    Args:
    file_path (str): The path to the .sql file.

    Returns:
    str: The contents of the .sql file ready to be processed by a duckdb connection.
    """
    file = Path(file_path)
    if not file.is_file() or file.suffix != '.sql':
        raise FileNotFoundError("The file does not exist or is not a .sql file.")
    sql = file.read_text()
    Markdown(f"```{sql}```")
    return sql


In [ ]:
def create_view(con, view_name, sql):
    con.sql(f"CREATE VIEW {view_name} AS {sql}")

In [ ]:
con = load_data_duckdb(data_url='../seeds/titanic3.csv')  # this CSV has home.dest renamed to home_dest

In [ ]:
df = con.sql(load_sql_query('../models/staging/renaming.sql').replace("raw.raw_titanic", "raw_titanic"))

In [ ]:
create_view(con, "staging_rename", load_sql_query('../models/staging/renaming.sql').replace("raw.raw_titanic", "raw_titanic"))

In [ ]:
con.sql("SHOW TABLES;")

In [ ]:
create_view(con, "staging_drop_cols", load_sql_query('../models/staging/dropping.sql').replace("titanic.staging.renaming", "staging_rename"))

In [ ]:
con.sql("SELECT * FROM staging_drop_cols").df()

In [ ]:
con.sql("""
WITH ConvertedAges AS (
    SELECT TRY_CAST(age AS FLOAT) AS age_float, *
    FROM raw_titanic
)
SELECT COUNT(age_float)
FROM ConvertedAges
WHERE age_float IS NULL;
""")

In [ ]:
df = con.sql("SELECT * FROM raw_titanic").df()

In [ ]:
df.dtypes

In [ ]:
df[df["age"].isna()]

In [ ]:
# con.close()

#### Pandas (comparison)

In [ ]:
pd_df = pd.read_csv(DATA_URL)

In [ ]:
df.equals(pd_df)

Check local version of file in `seeds` directory

In [ ]:
seeds_df = pd.read_csv("../seeds/titanic3.csv")

In [ ]:
seeds_df.rename(
    columns={"home_dest": "home.dest"}, inplace=True
)  # May need to change the column name from home.dest to home_dest to get tools to load

In [ ]:
seeds_df.equals(df)  # with this rename all good

In [ ]:
seeds_df.info()

In [ ]:
[col1 == col2 for col1, col2 in list(zip(seeds_df.columns.tolist(), df.columns.tolist()))]

In [ ]:
[type1 == type2 for type1, type2 in list(zip(seeds_df.dtypes.tolist(), df.dtypes.tolist()))]

In [ ]:
concatenated_df = pd.concat([df, seeds_df])
differences = concatenated_df.drop_duplicates(keep=False)

In [ ]:
len(differences) == 0

### SDF

https://www.sdf.com

In [ ]:
# QUESTION: How can I get the data out of SDF to reconcile?

### Profiling

In [ ]:
profile = ProfileReport(df)

In [ ]:
profile.to_notebook_iframe()

Now put the other dataset(s) age data on this plot too.

In [ ]:
# df_wikipedia = pd.read_html('https://en.m.wikipedia.org/wiki/Passengers_of_the_RMS_Titanic', header=0)

# Not working

In [ ]:
facts_tables = pd.read_html("http://www.titanicfacts.net/titanic-passenger-list.html", header=0)

In [ ]:
facts_tables[0].info()

In [ ]:
facts_tables[1].info()

In [ ]:
facts_tables[2].info()

So the "Titanic Facts" has the data spread across 3 tables with 324 (1st class), 284 (2nd class) and 709 (3rd class) passengers (1317 total) respectively. There is age data for all 1317 passengers. Contrast this with 1309 passengers in the Kaggle dataset [which does not claim to be complete -- in fact, somewhat disappointingly, there does not appear to be a reference for the data] with only 1046 age values.

In [ ]:
facts = pd.merge(pd.merge(facts_tables[0], facts_tables[1], how="outer"), facts_tables[2], how="outer")

In [ ]:
facts.to_csv("facts.csv")

In [ ]:
facts.info()

In [ ]:
facts.head()